In [72]:
import os
import re
import time
import numpy as np
import pandas as pd
from lxml import etree
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

In [78]:
# Configurations.
doi_url_pattern = "https://www.doi.org/{doi:s}"

# Load paper list requiring abstracts.
df = pd.read_csv("./no_abstracts.csv")
print(len(df))
df.head(2)

206


,type,abstract,booktitle,date-added,date-modified,doi,isbn,issn,keywords,pages,...,bdsk-url-1,author,journal,number,url,bdsk-url-2,archiveprefix,arxivid,eprint,pmid
0,article,NaN,NaN,2022-07-22 11:32:24 +1000,2022-07-22 11:33:04 +1000,10.1007/s11554-017-0716-1,NaN,18618200,NaN,415--418,...,http://link.springer.com/article/10.1007/s1155...,"[{'first': 'Sanghyun', 'last': 'Seo'}, {'first...",Journal of Real-Time Image Processing,3,http://link.springer.com/article/10.1007/s1155...,https://doi.org/10.1007/s11554-017-0716-1,NaN,NaN,NaN,NaN
1,article,NaN,NaN,2022-07-22 11:32:24 +1000,2022-07-22 11:33:06 +1000,10.1007/978-3-319-13695-0_71,NaN,NaN,NaN,697--705,...,http://link.springer.com/chapter/10.1007/978-3...,"[{'first': 'Samantha', 'last': 'Stout'}, {'fir...",Digital Heritage. Progress in Cultural Heritag...,NaN,http://link.springer.com/chapter/10.1007/978-3...,https://doi.org/10.1007/978-3-319-13695-0_71,NaN,NaN,NaN,NaN


In [79]:
# Start a webdriver instance.
driver = webdriver.Chrome("./chromedriver.exe")

In [80]:
# Iterating through papers.
for index, row in df.iterrows():
    doi = row.doi
    url = doi_url_pattern.format(doi=doi)
    print(url)
    driver.get(url)
    
    # Save html.
    with open("./html/{}.html".format(doi.replace("/", "_")), "w", encoding="utf-8") as f:
        f.write(driver.page_source)
    
    time.sleep(np.random.randint(5, 10))

https://www.doi.org/10.1007/s11554-017-0716-1
https://www.doi.org/10.1007/978-3-319-13695-0_71
https://www.doi.org/10.1007/978-3-030-76776-1_13
https://www.doi.org/10.1007/s00779-017-1069-2
https://www.doi.org/10.3103/s0005105516020059
https://www.doi.org/10.1007/s00779-017-1066-5
https://www.doi.org/10.1007/s10664-020-09806-x
https://www.doi.org/10.1007/s00146-021-01185-1
https://www.doi.org/10.1007/s11623-013-0213-z
https://www.doi.org/10.1007/s11623-015-0483-8
https://www.doi.org/10.1007/s40012-018-0198-8
https://www.doi.org/10.1007/s00521-017-3202-6
https://www.doi.org/10.1186/s42162-020-00113-9
https://www.doi.org/10.1007/s11107-015-0530-2
https://www.doi.org/10.1007/978-3-319-13695-0_17
https://www.doi.org/10.1007/s10639-018-09856-1
https://www.doi.org/10.1007/s11623-014-0211-9
https://www.doi.org/10.1007/978-3-030-77431-8_18
https://www.doi.org/10.1007/978-3-030-76776-1_4
https://www.doi.org/10.1007/978-3-662-45960-7_17
https://www.doi.org/10.1007/s11623-019-1215-2
https://www.d

In [81]:
# Extracting abstracts. 
for index, row in df.iterrows():
    doi = row.doi
    print(doi)
    with open("./html/{}.html".format(doi.replace("/", "_")), "r", encoding="utf-8") as f:
        html_raw = f.read()
    tree = etree.HTML(html_raw)
    abstract_str = ""
    
    # Iterating abstract paragraphs.
    for abs_node in tree.xpath("//div[@id='Abs1-content']//p"):
        if abs_node.text:
            abstract_str += abs_node.text
    print(len(abstract_str))
    df.loc[index, "abstract"] = abstract_str

10.1007/s11554-017-0716-1
0
10.1007/978-3-319-13695-0_71
654
10.1007/978-3-030-76776-1_13
1085
10.1007/s00779-017-1069-2
0
10.3103/s0005105516020059
658
10.1007/s00779-017-1066-5
0
10.1007/s10664-020-09806-x
0
10.1007/s00146-021-01185-1
472
10.1007/s11623-013-0213-z
692
10.1007/s11623-015-0483-8
402
10.1007/s40012-018-0198-8
925
10.1007/s00521-017-3202-6
0
10.1186/s42162-020-00113-9
0
10.1007/s11107-015-0530-2
0
10.1007/978-3-319-13695-0_17
635
10.1007/s10639-018-09856-1
0
10.1007/s11623-014-0211-9
586
10.1007/978-3-030-77431-8_18
1403
10.1007/978-3-030-76776-1_4
1297
10.1007/978-3-662-45960-7_17
1000
10.1007/s11623-019-1215-2
223
10.1007/s40012-016-0150-8
1186
10.1007/s40012-020-00289-1
1226
10.1007/s42979-020-00357-y
2068
10.1007/s11042-017-5154-3
0
10.1007/s41650-017-0039-1
911
10.1007/s10639-018-9805-4
0
10.3103/s0005105519060025
0
10.1007/s42045-020-00042-x
1543
10.1007/s11623-012-0226-z
418
10.1365/s40702-021-00773-5
928
10.1007/s42045-019-00018-6
1032
10.1007/s00287-019-01144-7


In [83]:
print("{} abstracts retrieved.".format(len(df) - df.abstract.isna().sum()))
df.head()

206 abstracts retrieved.


,type,abstract,booktitle,date-added,date-modified,doi,isbn,issn,keywords,pages,...,bdsk-url-1,author,journal,number,url,bdsk-url-2,archiveprefix,arxivid,eprint,pmid
0,article,,NaN,2022-07-22 11:32:24 +1000,2022-07-22 11:33:04 +1000,10.1007/s11554-017-0716-1,NaN,18618200,NaN,415--418,...,http://link.springer.com/article/10.1007/s1155...,"[{'first': 'Sanghyun', 'last': 'Seo'}, {'first...",Journal of Real-Time Image Processing,3,http://link.springer.com/article/10.1007/s1155...,https://doi.org/10.1007/s11554-017-0716-1,NaN,NaN,NaN,NaN
1,article,X-ray Fluorescence Spectroscopy was performed ...,NaN,2022-07-22 11:32:24 +1000,2022-07-22 11:33:06 +1000,10.1007/978-3-319-13695-0_71,NaN,NaN,NaN,697--705,...,http://link.springer.com/chapter/10.1007/978-3...,"[{'first': 'Samantha', 'last': 'Stout'}, {'fir...",Digital Heritage. Progress in Cultural Heritag...,NaN,http://link.springer.com/chapter/10.1007/978-3...,https://doi.org/10.1007/978-3-319-13695-0_71,NaN,NaN,NaN,NaN
2,article,Security and efficiency for resource constrain...,NaN,2022-07-22 11:32:24 +1000,2022-07-22 11:33:01 +1000,10.1007/978-3-030-76776-1_13,NaN,NaN,NaN,193--204,...,http://link.springer.com/chapter/10.1007/978-3...,"[{'first': 'Ansh', 'last': 'Tyagi'}, {'first':...","Computing Science, Communication and Security",NaN,http://link.springer.com/chapter/10.1007/978-3...,https://doi.org/10.1007/978-3-030-76776-1_13,NaN,NaN,NaN,NaN
3,article,,NaN,2022-07-22 11:32:24 +1000,2022-07-22 11:33:07 +1000,10.1007/s00779-017-1069-2,NaN,16174909,NaN,317--332,...,http://link.springer.com/article/10.1007/s0077...,"[{'first': 'Lachlan', 'last': 'Urquhart'}, {'f...",Personal and Ubiquitous Computing,2,http://link.springer.com/article/10.1007/s0077...,https://doi.org/10.1007/s00779-017-1069-2,NaN,NaN,NaN,NaN
4,article,The development trends of the global informati...,NaN,2022-07-22 11:32:24 +1000,2022-07-22 11:33:10 +1000,10.3103/s0005105516020059,NaN,0005-1055,NaN,47--55,...,http://link.springer.com/article/10.3103/S0005...,"[{'first': 'O.', 'middle': 'V.', 'last': 'Syun...",Automatic Documentation and Mathematical Lingu...,2,http://link.springer.com/article/10.3103/S0005...,https://doi.org/10.3103/s0005105516020059,NaN,NaN,NaN,NaN


In [84]:
# Exports.
df.to_csv("with_abstracts.csv", index=None)